In [ ]:
import pandas as pd
import numpy as np
import boto3
from datetime import datetime


## EDUCATION ##

# Chemin relatif vers le fichier education.csv
file_path = 'education-csv/education.csv'

# Lire le fichier en ignorant les premières lignes
df = pd.read_csv(file_path, skiprows=3)


# Supprimer la colonne "Indicator Code" qui semble être inutile
df.drop(columns=['Indicator Code'], inplace=True)

# Remplacer les noms de colonnes longs par des noms plus courts
new_columns = {
    'Country Name': 'Country',
    'Country Code': 'Code',
    'Indicator Name': 'Indicator'
}
df.rename(columns=new_columns, inplace=True)


# Filtrer les données pour obtenir celles spécifiques à Madagascar
madagascar_edu = df[df['Country'] == 'Madagascar']


## ECONOMY ##

# Chemin relatif vers le fichier pib.csv
file_path = 'economic-csv/pib.csv'

# Lire le fichier en ignorant les premières lignes
data = pd.read_csv(file_path, skiprows=3)


# Supprimer la colonne "Indicator Code" qui semble être inutile
data.drop(columns=['Indicator Code'], inplace=True)


# Remplacer les noms de colonnes longs par des noms plus courts
data.rename(columns=new_columns, inplace=True)


# Filtrer les données pour obtenir celles spécifiques à Madagascar
madagascar_pib = data[data['Country'] == 'Madagascar']


# Sélectionner les colonnes d'années pour l'éducation
edu_columns = madagascar_edu.columns[3:-1]  # Exclure les trois premières colonnes (Country, Code, et Indicator)
edu_data = madagascar_edu[edu_columns]

# Sélectionner les colonnes d'années pour l'économie
pib_columns = madagascar_pib.columns[3:-1]  # Exclure les trois premières colonnes (Country, Code, et Indicator)
pib_data = madagascar_pib[pib_columns]


# Extraire les valeurs des données d'éducation et d'économie pour Madagascar
edu_values = edu_data.iloc[0].values
pib_values = pib_data.iloc[0].values

# Créer un DataFrame commun avec les colonnes année et les valeurs de "edu" et "pib"
combined_data = pd.DataFrame({'Année': edu_columns,
                              'Taux d’achèvement de l’école primaire (%)': edu_values,
                              'Croissance du PIB (%)': pib_values})


# Supprimer les lignes avec des valeurs NaN dans combined_data
combined_data_cleaned = combined_data.dropna(how='any')

# Afficher le DataFrame final
print("Données d'éducation et d'économie pour Madagascar :")
print(combined_data_cleaned)


## UPLOAD TO S3 ##
# Nom du fichier de sortie avec la date actuelle
now = datetime.now()
output_filename = f"sujet_{now.strftime('%Y_%m_%d')}.csv"

# Clé S3 
s3_key = output_filename

# Sauvegarder le DataFrame au format CSV localement
combined_data_cleaned.to_csv(output_filename, index=False)

# Connexion à Amazon S3 via Boto
s3 = boto3.client('s3', aws_access_key_id='YOUR_ACCESS_KEY', aws_secret_access_key='YOUR_SECRET_KEY')

# Nom de votre bucket S3
bucket_name = 'data2exam'

# Stocker le fichier vers Amazon S3
s3.upload_file(output_filename, bucket_name, s3_key)

# Afficher un message de confirmation
print(f" \n Le fichier {output_filename} a été stocké avec succès vers {bucket_name}/{s3_key}")


